In [1]:
library(tidyverse)
library(DESeq2)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:lubridate’:

    intersect, setdiff, union


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’

In [2]:
preprocessed_list = qs::qread('../04_cleanup/preprocessed_list.qs')

In [3]:
deg_results_tibble = tibble()
for (name in c("0010_HY", "0010_HPF", "0010_SC", 
"0010_IC", "0010_PFC", "0037_HY", "0037_HPF", "0037_SC", "0037_IC", 
"0037_ZI")){
    seq_run = str_split(name, '_') %>% unlist %>% `[[`(1)
    region = str_split(name, '_') %>% unlist %>% `[[`(2)
    preprocessed = preprocessed_list[[name]]
    dds = preprocessed$dds
    deg_results = results(dds, contrast=c("diet","fast","chow")) %>% 
        data.frame %>%
        arrange(padj) %>%
#         filter(padj < 0.05) %>%
        mutate(seq_run = seq_run,
               region = region,
               comparison = 'fast.vs.chow') %>% 
        rownames_to_column(var = 'gene') %>%
    tibble
    deg_results_tibble = deg_results_tibble %>% rbind(deg_results)
    if (seq_run == '0010'){
        deg_results = results(dds, contrast=c("diet","HFD","chow")) %>% 
            data.frame %>%
            arrange(padj) %>%
    #         filter(padj < 0.05) %>%
            mutate(seq_run = seq_run,
                   region = region,
                   comparison = 'HFD.vs.chow') %>% 
            rownames_to_column(var = 'gene') %>%
        tibble
        deg_results_tibble = deg_results_tibble %>% rbind(deg_results)
    }
    
}

In [7]:
deg_summary = deg_results_tibble %>% filter(padj < 0.05) %>% group_by(seq_run, region, comparison) %>% summarise(n = n()) %>%
mutate(region = factor(region, levels=c('HY', 'HPF', 'SC', 'IC', 'PFC', 'ZI')),
       comparison = factor(comparison, levels=c("HFD.vs.chow", 'fast.vs.chow'))) %>%
arrange(seq_run, region, comparison)

deg_summary
deg_summary %>% write_csv('deg_summary.csv')

`summarise()` has grouped output by 'seq_run', 'region'. You can override using
the `.groups` argument.


seq_run,region,comparison,n
<chr>,<fct>,<fct>,<int>
0010,HY,HFD.vs.chow,2
0010,HY,fast.vs.chow,37
0010,HPF,HFD.vs.chow,129
0010,HPF,fast.vs.chow,6
0010,SC,fast.vs.chow,7
0010,PFC,fast.vs.chow,11
0037,HY,fast.vs.chow,2728
0037,HPF,fast.vs.chow,1019
0037,SC,fast.vs.chow,1612
